Go through the training images in the tfrecords file and write them to jpegs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.misc import imsave
import os
%matplotlib inline

C:\Users\eric\Anaconda2\envs\exts-aml2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# make sure everything was written properly by reading it back out
def read_and_decode_single_example(filenames):
    filename_queue = tf.train.string_input_producer(filenames, num_epochs=1)
    
    reader = tf.TFRecordReader()
    
    _, serialized_example = reader.read(filename_queue)
    # The serialized example is converted back to actual values.
    # One needs to describe the format of the objects to be returned
    features = tf.parse_single_example(
        serialized_example,
        features={
            # We know the length of both fields. If not the
            # tf.VarLenFeature could be used
            'label': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        })
    
    # now return the converted data
    label = features['label']
    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.reshape(image, [299, 299, 1])
    
    # scale the image
    #image = tf.image.per_image_standardization(image)
    
    # random flip image
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    #image = tf.image.random_brightness(image, max_delta=10)
    #image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
    
    return label, image

In [3]:
label, image = read_and_decode_single_example(["data/training9_0.tfrecords", "data/training9_1.tfrecords", "data/training9_2.tfrecords", "data/training9_3.tfrecords", "data/training9_4.tfrecords"])

images_batch, labels_batch = tf.train.batch([image, label], batch_size=64, capacity=2000)

global_step = tf.Variable(0, trainable=False)

In [10]:
dest_dir = os.path.join("data", "images_9")
if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)
    os.mkdir(os.path.join(dest_dir, "0"))
    os.mkdir(os.path.join(dest_dir, "1"))
    os.mkdir(os.path.join(dest_dir, "2"))
    os.mkdir(os.path.join(dest_dir, "3"))
    os.mkdir(os.path.join(dest_dir, "4"))

# how many total images are there?
total_images = 43739
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    counter = 0
    while counter < total_images:
        la_b, im_b = sess.run([labels_batch, images_batch])
        
        # go through each image in the batch and save it to a directory depending on its class
        for image, label in zip(im_b, la_b):
            # create the filename from the counter so each image has a distinct name
            filename = str(counter) + ".jpg"
            
            # put the image in a directory according to its label
            image_dir = os.path.join(dest_dir, str(label))
            
            # reshape the image
            image = image.reshape((299,299))
            
            # save the image
            imsave(os.path.join(image_dir, filename), image)

            counter += 1
    coord.request_stop()
    
    # Wait for threads to stop
    coord.join(threads)

21.36877892356526
10.128382425740336
28.025339230147654
9.905500460933276
18.264761206016594
19.723216367614473
5.8048971316422655
19.697080951314597
9.946353165740446
14.28138557133216
52.20470672100469
36.3059797838363
5.021630055859638
18.320553935578513
26.842363800778617
57.45264177362011
11.274690484858693
7.660515245444074
28.25892250802674
17.277403005520338
11.342811799978698
8.468497131106833
7.957438173038621
19.91439986489477
31.058340784757387
15.447608924652489
7.419835351865077
14.696458918454104
5.919608277187264
14.841844284976029
4.827517571664751
6.4597691032888855
17.161402392911278
7.80047175846602
25.057141345223826
18.465645139315967
21.556882193007507
30.261413261671112
10.14056357937036
11.776938247675652
22.61535187922285
9.350637884598386
14.355089659166648
5.7694328321569825
32.55398199982781
35.13993491274338
11.665819592265846
11.723168603476479
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>,

KeyboardInterrupt: 